In [21]:
import numpy as np
import tensorflow as tf
from functools import reduce
import matplotlib.pyplot as plt

In [22]:
num_epochs = 1000000
total_series_length = 50000
train_sample_size = 10000
truncated_backprop_length = 15
state_size = 4
echo_step = 3
num_of_operations = 3
#num_batches = total_series_length//batch_size//truncated_backprop_length
num_layers = 3
num_of_steps = 5
num_features = 3
num_samples = 10
max_num_features = 16
max_num_outputs = 16
batch_size = num_samples
def np_add(vec):
    return reduce((lambda x, y: x + y),vec)

def samples_generator(fn, shape, rng, seed=None):
    '''
    Generate random samples for the model:
    @fn - function to be applied on the input features to get the ouput
    @shape - shape of the features matrix (num_samples, num_features)
    @rng - range of the input features to be generated within (a,b)
    @seed  - generation seed
    Outputs a tuple of input and output features matrix
    '''
    x = (rng[1] - rng[0]) * np.random.random_sample(shape) + rng[0]
    y = np.apply_along_axis(fn, 1, x).reshape((-1,1))

    #x = x.reshape((batch_size, -1))  # The first index changing slowest, subseries as rows
    #y = y.reshape((batch_size, -1))

    return (x, y)

In [23]:
#x,y = samples_generator(add, (num_samples, num_features) , (0, 100))

In [ ]:
def tf_multiply(inpt):
    return tf.reshape( tf.reduce_prod(inpt, axis = 1), [batch_size, -1])

def tf_add(inpt):
    return  tf.reshape( tf.reduce_sum(inpt, axis = 1), [batch_size, -1])

def tf_stall(a):
    return a

'''
def tf_divide(inpt):
    return tf.einsum('i,j->ij',inpt)

def tf_substract(x,y):
    return tf.subtract(x,y)
'''

'''
function_dict = {
    1   : tf_multiply,
    2   : tf_add,
    3   : tf_divide,
    4   : tf_substract,
    5   : tf_retain    
}
'''
batchX_placeholder = tf.placeholder(tf.float32, [batch_size, None], name="batchX")
batchY_placeholder = tf.placeholder(tf.float32, [batch_size, None], name="batchY")

init_state = tf.placeholder(tf.float32, [batch_size, state_size], name="init_state")

W = tf.Variable(np.random.rand(state_size+num_features, state_size), dtype=tf.float32, name="W")
b = tf.Variable(np.zeros((state_size)), dtype=tf.float32, name="b")

W2 = tf.Variable(np.random.rand(state_size, num_of_operations),dtype=tf.float32, name="W2")
b2 = tf.Variable(np.zeros((num_of_operations)), dtype=tf.float32, name="b2")

dummy_matrix = tf.zeros([batch_size, num_features], tf.float32, name="dummy_constant")

#inputs_series = tf.unstack(batchX_placeholder, axis=1)
#labels_series = tf.unstack(batchY_placeholder, axis=1)


# Forward pass
#current_state = init_state
#states_series = []

#output = tf.reshape(current_input, [batch_size, 1])

current_state = init_state

output = batchX_placeholder

outputs = []

for timestep in range(num_of_steps):
    print("timestep " + str(timestep))
    current_input = output
        
    
    
    input_and_state_concatenated = tf.concat([current_input, current_state], 1)  # Increasing number of columns
    next_state = tf.tanh(tf.matmul(input_and_state_concatenated, W) + b)  # Broadcasted addition
    current_state = next_state
    
    #calculate softmax and produce the mask of operations
    logits = tf.matmul(next_state, W2) + b2 #Broadcasted addition
    softmax = tf.nn.softmax(logits)
    #argmax = tf.argmax(softmax, 1)
    '''
    print(logits)
    print(softmax)
    print(argmax)
    '''
    #perform ops
    add   = tf_add(current_input)
    mult  = tf_multiply(current_input)
    stall = tf_stall(current_input)
    #add = tf.reshape( tf.reduce_prod(current_input, axis = 1), [batch_size, -1])
    #mult = tf.reshape( tf.reduce_sum(current_input, axis = 1), [batch_size, -1])
    #stall = current_input
    #values = tf.concat([add, mult, stall], 1)
    values = tf.concat([add, mult, stall], 1)
    values = tf.cast(values,tf.float32)
    #get softmaxes for operations
    #add_softmax = tf.slice(softmax, [0,0], [batch_size,1])
    #mult_softmax = tf.slice(softmax, [0,1], [batch_size,1])
    #stall_softmax = tf.slice(softmax, [0,2], [batch_size,1])
    #produce output matrix
    #onehot  = tf.one_hot(argmax_dum, num_of_operations)
    #stall_width = tf.shape(stall)[1]
    #stall_select = tf.slice(onehot, [0,2], [batch_size,1])
    #mask_arr = [onehot]
    #for i in range(num_features-1):
    #    mask_arr.append(stall_select)
    #mask = tf.concat(mask_arr, 1)
    #argmax = tf.reshape( softmax, [batch_size, -1])
    #mask = onehot
    #mask = tf.cast(mask, tf.float32)
    #mask = tf.cast(mask, tf.bool)
    #apply mask
    #output = tf.boolean_mask(values,mask)
    
    #in the train mask = saturated softmax for all ops. in test change it to onehot(hardmax)
    add_softmax   = tf.slice(softmax, [0,0], [batch_size,1])
    mult_softmax  = tf.slice(softmax, [0,1], [batch_size,1])
    stall_softmax = tf.slice(softmax, [0,2], [batch_size,1])

    add_width   = tf.shape(add)[1]
    mult_width  = tf.shape(mult)[1]
    stall_width = tf.shape(stall)[1]


    add_final   = add * add_softmax
    mult_final  = mult * mult_softmax
    stall_final = stall * stall_softmax

    ##conact add and mult results with zeros matrix
    add_final = tf.concat([add_final, tf.slice(dummy_matrix, [0,0], [batch_size, num_features - add_width])], 1) 
    mult_final = tf.concat([mult_final, tf.slice(dummy_matrix, [0,0], [batch_size, num_features - mult_width])], 1) 

    output = add_final + mult_final + stall_final
    outputs.append(output)
    #output = tf.reshape( tf.reduce_sum(output, axis = 1), [batch_size, -1])
    #print(select_add)
    #print(select_mult)
    #print(select_stall)
    
    #output
    #output = select_add
    
    '''
    output = tf.case({
                   tf.equal(argmax, tf.constant([0], tf.int64)): lambda: ,
                   tf.equal(argmax, tf.constant([1], tf.int64)): lambda:},
                   default= lambda: ,
                   exclusive=True)
    '''
    #

    #states_series.append(next_state)
    
    #print(current_state)

#logits_series = [tf.matmul(state, W2) + b2 for state in states_series] #Broadcasted addition
#predictions_series = [tf.nn.softmax(logits) for logits in logits_series]
    

#losses = [tf.nn.sparse_softmax_cross_entropy_with_logits(logits = logits, labels = labels) for logits, labels in zip(logits_series,labels_series)]

math_error = 0.5 * tf.square(tf.subtract(tf.reduce_sum(output, axis = 1) , batchY_placeholder))

total_loss = tf.reduce_sum(math_error)

tf.trainable_variables()
grads = tf.gradients(total_loss, [W,b,W2,b2], name="gradients")
train_step = tf.train.AdamOptimizer(0.01).minimize(total_loss)
print("grads are")
print(grads)

timestep 0
timestep 1
timestep 2
timestep 3
timestep 4
grads are
[<tf.Tensor 'gradients_8/AddN_16:0' shape=(7, 4) dtype=float32>, <tf.Tensor 'gradients_8/AddN_15:0' shape=(4,) dtype=float32>, <tf.Tensor 'gradients_8/AddN_14:0' shape=(4, 3) dtype=float32>, <tf.Tensor 'gradients_8/AddN_12:0' shape=(3,) dtype=float32>]


In [ ]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    #plt.ion()
    #plt.figure()
    #plt.show()
    loss_list = []

    for epoch_idx in range(num_epochs):
        x,y = samples_generator(np_add, (num_samples, num_features) , (0, 100))
        _current_state = np.zeros((batch_size, state_size))

        #print("\r New data, epoch", epoch_idx)

        #for batch_idx in range(num_batches):
            #start_idx = batch_idx * truncated_backprop_length
            #end_idx = start_idx + truncated_backprop_length

        #batchX = x[:,start_idx:end_idx]
        #batchY = y[:,start_idx:end_idx]

        _total_loss, _train_step, _current_state, _outputs = sess.run(
                [total_loss, train_step, current_state, outputs],
                feed_dict={
                    init_state:_current_state,
                    batchX_placeholder:x,
                    batchY_placeholder:y
                })
        loss_list.append(_total_loss)
            
        print("Epoch",epoch_idx, "Loss", _total_loss , "    ", end="\r")